In [ ]:
import polars as pl
import networkx as nx
import rtsvg
import time
import os

rt = rtsvg.RACETrack()

dfs = []

_relates_    = [('fm', 'to')]
_files_      = os.listdir('../../data/stanford/facebook')
_edge_files_ = [_file_ for _file_ in _files_ if _file_.endswith('.edges')]
for _file_ in _edge_files_:
    with open('../../data/stanford/facebook/' + _file_, 'rt') as f:
        _edges_ = f.read()
        _lu_ = {'fm':[], 'to':[]}
        for _edge_ in _edges_.split('\n'):
            if _edge_ == '': continue
            _lu_['fm'].append(_edge_.split(' ')[0])
            _lu_['to'].append(_edge_.split(' ')[1])
        dfs.append(pl.DataFrame(_lu_))

_stats_ = {'nodes':[], 'edges':[], 'time_spring_layout':[], 'time_dijkstra':[]}
for _df_ in dfs:
    g     = rt.createNetworkXGraph(_df_, _relates_)
    t0    = time.time()
    pos   = nx.spring_layout(g)
    t1    = time.time()
    dists = dict(nx.all_pairs_dijkstra_path_length(g))
    t2    = time.time()

    _stats_['nodes']             .append(len(g.nodes()))
    _stats_['edges']             .append(len(g.edges()))
    _stats_['time_spring_layout'].append(t1 - t0)
    _stats_['time_dijkstra']     .append(t2 - t1)

df = pl.DataFrame(_stats_)
rt.table([rt.xy(df, x_field='nodes', y_field='time_spring_layout'),
          rt.xy(df, x_field='nodes', y_field='time_dijkstra'),
          rt.xy(df, x_field='edges', y_field='time_spring_layout'),
          rt.xy(df, x_field='edges', y_field='time_dijkstra')], per_row=2)